In [1]:
import os
import sys

def adding_module_path():
    module_path = os.path.abspath(os.path.join('..'))

    if module_path not in sys.path:
        sys.path.append(module_path)

adding_module_path()

In [2]:
from src.data_preparing.build.gutenberg_builder import GutenbergBuilder
from src.app.project_setup import ProjectSetup
from src.data_preparing.split.run_split_deps_on_stats import run_split_deps_on_stats_same_dir
from src.utils.create_path_to_gutenberg import create_path_to_gutenberg_sentence_authors_sentence, create_path_to_gutenberg_authors
from src.utils.create_test_dataset_from import create_test_dataset_from
from src.data_preparing.split.run_split_deps_on_stats import run_split_deps_on_stats_same_dir
from src.utils.create_path_to_gutenberg import create_path_to_gutenberg_sentence_authors_sentence
from src.config.config import PATH_TO_DATASET_FOLDER_TEST, PATH_TO_DATASET_FOLDER_TEST, AUTHORS_FILE_NAME

from src.tokenizers.prepare_dataset_from_tokenizer import prepare_dataset_from_tokenizer
from src.tokenizers.transformer_tokenizer import TransformerTokenizer
import tensorflow as tf
from src.encoder.create_encoder_from_path import create_encoder_from_path
from src.testing.get_testing_dataset import dataset as test_dataset
from src.data_loading.get_dataset_object_from import get_dataset_object_from_path
from src.utils.create_path_to_gutenberg import get_paths_to_gutenberg

In [3]:
from transformers import TFAutoModel
from src.experiments.settings import LearningSettings

In [4]:
settings = LearningSettings()

In [5]:

model_name = "bert-base-uncased"

In [6]:

path_data, path_authors = get_paths_to_gutenberg(10, 3, PATH_TO_DATASET_FOLDER_TEST)
dataset = get_dataset_object_from_path(path_data, ';', None)


# tokenizer = TransformerTokenizer(
#     model_name, 
#     create_encoder_from_path(
#         path_authors
#     )
# )

tokenizer = TransformerTokenizer(
    model_name, 
    None
)


In [7]:
transformer = TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [8]:
input_ids = tf.keras.layers.Input(shape=(512, ), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512, ), name='attention_mask', dtype='int32')

embeddings = transformer(input_ids, attention_mask=mask)[0]  # we only keep tensor 0 (last_hidden_state)

X = tf.keras.layers.GlobalMaxPool1D()(embeddings)  # reduce tensor dimensionality
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.1)(X)
y = tf.keras.layers.Dense(10, activation='softmax', name='outputs')(X)  # adjust based on number of sentiment classes

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

In [9]:
model.layers[2].trainable = False

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [11]:
model.compile(loss=settings.loss, optimizer=settings.optimizer, metrics=settings.metric)

In [12]:
settings.batch_size

32

In [13]:
settings.epochs

1

In [15]:
history = model.fit(prepare_dataset_from_tokenizer(test_dataset, tokenizer), epochs=2)

Epoch 1/2


In [ ]:
test = model.predict(dataset)

In [14]:

for x in prepare_dataset_from_tokenizer(test_dataset, tokenizer).batch(32):
    print(x)
    break

({'input_ids': <tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[ 101, 7592, 1045, ...,    0,    0,    0],
       [ 101, 1045, 2066, ...,    0,    0,    0],
       [ 101, 3231, 2182, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(3,), dtype=int32, numpy=array([12, 15, 20])>)


Run model experiment with to lower

... find best preprocessing

# Test ideas

- one dataset - combination of preprocessing
- size of input (k) - deps on bert - make statistics
- type of transformer - distilbert, larbe bert etc
- head of transformer - 
- encoding of transformer
- classic model of tranformer
- testování proti Glove LSTM, Embedding Dense, Glove LSTM, Glove Dense
